In [24]:
""" 누락된 값이 있는 테이블 형성 """

import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0'''

df = pd.read_csv(StringIO(csv_data))
# csv_data에 저장된 문자열을 판다스 DataFrame으로 변환
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [25]:
""" 누락된 값 확인하기 """

df.isnull().sum()
# isnull()은 null값이 들어있으면 True
# isnull().sum()을 통해 각 열에 NULL값 개수를 파악할 수 있다.

# df.values를 통해 DataFrame의 values 속성을 사용해 넘파이 배열을 얻을 수 있다.

A    0
B    0
C    1
D    1
dtype: int64

In [26]:
""" 누락된 값을 행 통째로 제거하기"""

df.dropna(axis = 0)
# NULL값이 하나라도 있는 행을 모두 제거한다.

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [27]:
""" 누락된 값을 열 통째로 제거하기 """

df.dropna(axis = 1)
# NULL값이 하나라도 있는 열을 모두 제거한다.

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [28]:
""" 모든 행이 NULL일 때만 제거하기"""

df.dropna(how = 'all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [29]:
""" 행의 실수 값 개수에 따라 제거하기"""

df.dropna(thresh = 4)
# 실수 값이 네 개보다 작은 행을 삭제한다.

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [30]:
""" 특정 열에 NaN(NULL)이 있는 행만 삭제"""

df.dropna(subset = ['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


In [37]:
""" 전체 평균으로 대체하기
책 내용은 부분적으로 아래와 같으나,
사이킷런 버전 패치에 의해 Imputer 클래스가 삭제되었음.
https://scikit-learn.org/stable/modules/impute.html#impute

from sklearn.preprocessing import Imputer
imr = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
"""
import numpy as np
from sklearn.impute import SimpleImputer
imr = SimpleImputer(missing_values = np.nan, strategy = 'mean')
# mean 말고 median, most_frequent, constant 등이 들어갈 수 있다.
# constant일 때, fill_value 매개변수에 채우려는 값을 지정한다.
# axis = 1을 계산하려는 경우, 즉, 행 계산을 하고자 하는 경우, FunctionTransformer 를 사용하여
# 입력 행열의 행과 열을 바꾸어 대체할 값을 계산한 후 다시 변환된 행렬의 열과 행을 바꾼다.
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [41]:
""" FunctionTransformer로 누락 대체 행 계산하기"""
from sklearn.preprocessing import FunctionTransformer
simr = imr
ftr_simr = FunctionTransformer(lambda X: simr.fit_transform(X.T).T, validate = False)
imputed_data2 = ftr_simr.fit_transform(df.values)
imputed_data2

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])